In [439]:
#imports
import pandas as pd 
from slugify import slugify
import requests
from tqdm.notebook import tqdm
from time import sleep
from bs4 import BeautifulSoup as bs

In [440]:
#getting both csv files 
movie_details = pd.read_csv('/Users/armaan/src/bbox/mdata/thenum/movie_details.csv') 
imdb_movies = pd.read_csv('/Users/armaan/src/bbox/mdata/imdb/imdb_movies.csv') 
#converting names to lower and some cols to int
movie_details['name_lower'] = movie_details['name'].str.lower()
imdb_movies['year'] = pd.to_numeric(imdb_movies.startYear, errors='coerce')
imdb_movies['primaryTitle_lower'] = imdb_movies['primaryTitle'].str.lower()
imdb_movies['runtimeMinutes'] = pd.to_numeric(imdb_movies.runtimeMinutes, errors='coerce')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [441]:
#normalization
def normalize(x):
    if x:
        #for star wars
        x = x.replace('-ep-', '-episode-')
        #for harry potter
        x = x.replace('part-ii', 'part-2')
        x = x.replace('part-i', 'part-1')
        #for e.t. only!
        if x.startswith('et-'):
            x = x.replace('et-', 'e-t-')
        #for rise of skywalker only!
        if x == 'star-wars-the-rise-of-skywalker':
            x = "star-wars-episode-ix-the-rise-of-skywalker"
    return x

#find duplicates
def duplicate_scraper(imdb_id):
    r = requests.get("https://www.imdb.com/title/{}/".format(imdb_id))
    soup = bs(r.text)
    if soup.find("h3",{"class": "subheading"}).string == "Box Office":
        drop = False
    else:
        drop = True 
    return drop

In [442]:
#sluggifying and normalizing all names
movie_details['name_slug'] = movie_details.name_lower.apply(slugify)
imdb_movies['name_slug'] = imdb_movies.primaryTitle_lower.apply(slugify)
movie_details['name_slug'] = movie_details.name_slug.apply(normalize)
#creating new df with updated values
new_df = pd.merge(movie_details, imdb_movies, how ='inner', right_on = ['name_slug', 'year'], left_on = ['name_slug', 'year'])

In [443]:
#find duplicates and get rid of them
duplicates = new_df[new_df.name.duplicated()]
for index, row in duplicates.iterrows():
    match = new_df[(new_df['name'] == row["name"]) & (new_df['year'] == row["year"])]
    if len(match) > 1:
        imdb_ids = match["tconst"]
        for imdb_id in imdb_ids:
            drop = duplicate_scraper(imdb_id)
            if drop == True:
                new_df.drop( new_df[ new_df['tconst'] == imdb_id ].index , inplace=True)

In [444]:
new_df.drop(["endYear", "isAdult", "titleType", "startYear", "primaryTitle_lower", "runtimeMinutes", "genres", "name_lower", "name_slug", "primaryTitle", "originalTitle"], axis=1, inplace = True)
new_df

,name,year,domestic_box_office,international_box_office,worldwide_box_office,opening_weekend,legs,production_budget,theater_counts,mpaa_rating,running_time,genre,production_method,creative_type,production_companies,synopsis,link,tconst
0,Star Wars Ep. VII: The Force Awakens,2015,936662225,1131561399,2068223624,247966675,3.78,306000000,9.2,PG-13,136,Adventure,Animation/Live Action,Science Fiction,Lucasfilm,"Rey, a scavenger from the planet Jakku, and re...",https://www.the-numbers.com/movie/Star-Wars-Ep...,tt2488496
1,Avengers: Endgame,2019,858373000,1939427564,2797800564,357115007,2.40,400000000,8.1,PG-13,181,Action,Animation/Live Action,Super Hero,Marvel Studios,The grave course of events set in motion by Th...,https://www.the-numbers.com/movie/Avengers-End...,tt4154796
2,Avatar,2009,760507625,2028193712,2788701337,77025481,9.87,237000000,13.2,PG-13,162,Action,Animation/Live Action,Science Fiction,Dune Entertainment,"Jake Sully is a wounded ex-marine, thrust into...",https://www.the-numbers.com/movie/Avatar#tab=s...,tt0499549
3,Black Panther,2018,700059566,646853595,1346913161,202003951,3.47,200000000,9.9,PG-13,120,Action,Live Action,Super Hero,Marvel Studios,King Txc3xa2xc2x80xc2x99Challa returns home to...,https://www.the-numbers.com/movie/Black-Panthe...,tt1825683
4,Avengers: Infinity War,2018,678815482,1369544272,2048359754,257698183,2.63,300000000,7.7,PG-13,156,Action,Animation/Live Action,Super Hero,Marvel Studios,As the Avengers and their allies have continue...,https://www.the-numbers.com/movie/Avengers-Inf...,tt4154756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,Harry Potter and the Order of the Phoenix,2007,292137260,650939197,943076457,77108414,3.79,150000000,6.1,PG-13,138,Adventure,Animation/Live Action,Fantasy,Heyday Films,NaN,https://www.the-numbers.com/movie/Harry-Potter...,tt0373889
98,"The Chronicles of Narnia: The Lion, the Witch ...",2005,291710957,428828615,720539572,65556312,4.45,180000000,8.7,PG,140,Adventure,Animation/Live Action,Fantasy,Walt Disney Pictures,Four young adventurers playing hide and-seek i...,https://www.the-numbers.com/movie/Chronicles-o...,tt0363771
99,Man of Steel,2013,291045518,376954000,667999518,116619362,2.50,225000000,5.0,PG-13,142,Action,Live Action,Super Hero,Warner Bros.,A young boy learns that he has extraordinary p...,https://www.the-numbers.com/movie/Man-of-Steel...,tt0770828
100,Star Wars Ep. V: The Empire Strikes Back,1980,290271960,257697044,547969004,4910483,13.21,23000000,13.1,PG,124,Adventure,Live Action,Science Fiction,Lucasfilm,NaN,https://www.the-numbers.com/movie/Star-Wars-Ep...,tt0080684


In [445]:
#downloads df to csv
new_df.to_csv('/Users/armaan/src/bbox/mdata/master_table.csv', index=False)